# Managing feature versions in SageMaker Feature Store

In [ ]:
import boto3, sagemaker
import pandas as pd
from sagemaker.session import Session

sagemaker_session = Session()
region = sagemaker_session.boto_region_name
boto_session = boto3.Session(region_name=region)
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket()
prefix = 'amazon-reviews-featurestore'

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group_name = # feature group name
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

In [ ]:
feature_group_query = feature_group.athena_query()
feature_group_table = feature_group_query.table_name

print(feature_group_table)

## Reading a single record from the online feature store

In [ ]:
record_identifier_value = 'R38H3UO1J190GI'

response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString=record_identifier_value
    # FeatureNames=['review_body']
)

record = response['Record']
record

## Add a newer version of a record to the online feature store

In [ ]:
import time

for feature in record:
    if (feature['FeatureName']) == 'star_rating':
        feature['ValueAsString'] = '4'
    if (feature['FeatureName']) == 'event_time':
        feature['ValueAsString'] = str(int(round(time.time())))
        
record

In [ ]:
response = featurestore_runtime.put_record(
    FeatureGroupName=feature_group_name,
    Record=record)

response

In [ ]:
response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString=record_identifier_value)

record = response['Record']
record

## Query the offline store
... once it's been updated. This could take a few minutes :)

In [ ]:
query_string = 'SELECT review_id,event_time,star_rating,is_deleted FROM "'+ feature_group_table+'"' + ' WHERE review_id=\''+record_identifier_value+'\';'

query_string

In [ ]:
records = pd.DataFrame()
feature_group_query.run(query_string=query_string, output_location='s3://'+default_bucket+'/query_results/')
feature_group_query.wait()
records = feature_group_query.as_dataframe()
records.head()

## Time-travel in the offline store

In [ ]:
# update timestamp :)

query_string = 'SELECT * FROM "'+ feature_group_table+'"' + \
' WHERE review_id=\''+record_identifier_value+'\' AND event_time < 1616154941 AND NOT is_deleted LIMIT 10;'

query_string

In [ ]:
feature_group_query.run(query_string=query_string, output_location='s3://'+default_bucket+'/query_results/')
feature_group_query.wait()
records = feature_group_query.as_dataframe()
records.head()

## Delete a single record from the online store

In [ ]:
response = featurestore_runtime.delete_record(
    FeatureGroupName=feature_group_name,
    RecordIdentifierValueAsString=record_identifier_value,
    EventTime=str(int(round(time.time())))
 )

response

In [ ]:
response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString=record_identifier_value)

response

In [ ]:
query_string = 'SELECT * FROM "'+ feature_group_table+'"' + ' WHERE review_id=\''+record_identifier_value+'\';'

query_string

In [ ]:
feature_group_query.run(query_string=query_string, output_location='s3://'+default_bucket+'/query_results/')
feature_group_query.wait()
records = feature_group_query.as_dataframe()
records.head()